I intend to reuse the Alexnet thathas been created [here](https://github.com/Evolving-AI-Lab/deep_learning_for_camera_trap_images)

In [1]:
import numpy as np
import tensorflow as tf

import os, sys, time
sys.path.append('/Users/manishrai/Desktop/UMN/Research/Zooniverse/deep_learning_for_camera_trap_images/phase2/architectures')
sys.path.append('/Users/manishrai/Desktop/UMN/Research/Zooniverse/deep_learning_for_camera_trap_images/phase2/')
sys.path.append('/Users/manishrai/Desktop/UMN/Research/Zooniverse/Code/Data_gathering')

In [2]:
import ImageToArray, cnn_utils
import common, alexnet

Dataset creation

In [3]:
startTime = time.time()
# directory of the training dataset
train_data_dog_dir = '/Users/manishrai/Desktop/UMN/Research/Zooniverse/Data/CatsVsDogs_test/train/dogs/'
train_data_cat_dir = '/Users/manishrai/Desktop/UMN/Research/Zooniverse/Data/CatsVsDogs_test/train/cats/'

X_train_dogs = ImageToArray.get_image_to_array(train_data_dog_dir, w = 128, h = 128)
X_train_cats = ImageToArray.get_image_to_array(train_data_cat_dir, w = 128, h = 128)
Y_train_dogs = np.ones((1, X_train_dogs.shape[0]), dtype=np.int8)
Y_train_cats = np.zeros((1, X_train_cats.shape[0]), dtype=np.int8)


# Append and permuate X and Y
X_train_orig =  np.concatenate((X_train_dogs, X_train_cats), axis = 0 )
m = X_train_orig.shape[0] # number of training examples
Y_train_orig =  np.concatenate((Y_train_dogs, Y_train_cats), axis = 1 ).reshape(m, 1)

# Randomly shuffle the arrays
shuffle_index_train = np.random.permutation(range(m))
X_train_orig_p = X_train_orig[shuffle_index_train]
Y_train_orig_p = Y_train_orig[shuffle_index_train]

# Normalize the image arrays
X_train_orig_norm = ImageToArray.get_normalized_pixels(X_train_orig_p)
endTime = time.time()


print('Took %s seconds to calculate.' % (endTime - startTime))
print('memory usage for training data - %s' % X_train_dogs.nbytes)

Took 55.6005470753 seconds to calculate.
memory usage for training data - 690487296


Convert to tensors

In [4]:
X_train = tf.convert_to_tensor(X_train_orig_p, dtype = "float32")
Y_train = tf.convert_to_tensor(Y_train_orig_p)

In [5]:
X_train.get_shape, Y_train.get_shape

(<bound method Tensor.get_shape of <tf.Tensor 'Const:0' shape=(3560, 128, 128, 3) dtype=float32>>,
 <bound method Tensor.get_shape of <tf.Tensor 'Const_1:0' shape=(3560, 1) dtype=int8>>)

In [20]:
tf.reset_default_graph()
g = tf.Graph()
with tf.Session() as sess:
    with g.as_default():
        alexnet.inference(X_train, 2, 0.00, 0.5, None, transfer_mode= False)

INFO:tensorflow:Scale of 0 disables regularizer.


ValueError: Tensor("Const:0", shape=(3560, 128, 128, 3), dtype=float32) must be from the same graph as Tensor("conv1/weights:0", shape=(11, 11, 3, 64), dtype=float32_ref, device=/device:CPU:0).

In [21]:
def resize_image(image, target_size):
    """ Resize Image """
    image = tf.image.resize_images(image, size=target_size)
    image = tf.divide(image, 255.0)
    return image

In [22]:
image = '/Users/manishrai/Desktop/UMN/Research/Zooniverse/Data/CatsVsDogs_test/train/dogs/dog.1.jpg'

In [42]:
sess = tf.Session()
image = resize_image(X_train_dogs[1], [224, 224]) # The image data has to be in an array format
#sess.close()

In [43]:
def _crop(image, offset_height, offset_width, crop_height, crop_width):
    """Crops the given image using the provided offsets and sizes.
    Note that the method doesn't assume we know the input image size but it does
    assume we know the input image rank.
    Args:
    image: an image of shape [height, width, channels].
    offset_height: a scalar tensor indicating the height offset.
    offset_width: a scalar tensor indicating the width offset.
    crop_height: the height of the cropped image.
    crop_width: the width of the cropped image.
    Returns:
    the cropped (and resized) image.
    Raises:
    InvalidArgumentError: if the rank is not 3 or if the image dimensions are
      less than the crop size.
    """
    original_shape = tf.shape(image)

    rank_assertion = tf.Assert(
      tf.equal(tf.rank(image), 3),
      ['Rank of image must be equal to 3.'])
    with tf.control_dependencies([rank_assertion]):
        cropped_shape = tf.stack([crop_height, crop_width, original_shape[2]])

    size_assertion = tf.Assert(
        tf.logical_and(
          tf.greater_equal(original_shape[0], crop_height),
          tf.greater_equal(original_shape[1], crop_width)),
        ['Crop size greater than the image size.'])

    offsets = tf.to_int32(tf.stack([offset_height, offset_width, 0]))

    # Use tf.slice instead of crop_to_bounding box as it accepts tensors to
    # define the crop size.
    with tf.control_dependencies([size_assertion]):
        image = tf.slice(image, offsets, cropped_shape)
    return tf.reshape(image, cropped_shape)

In [47]:
sess.run(_crop(image, 100, 100, 100, 100))

array([[[ 0.63017124,  0.54053545,  0.41208398],
        [ 0.61864728,  0.52901137,  0.39991963],
        [ 0.5823127 ,  0.49235675,  0.36374524],
        ..., 
        [ 0.62000805,  0.38303334,  0.1611046 ],
        [ 0.63377327,  0.3973588 ,  0.17238878],
        [ 0.63217258,  0.39799902,  0.16278481]],

       [[ 0.50212103,  0.41472608,  0.27442992],
        [ 0.57895166,  0.4915567 ,  0.34805927],
        [ 0.59895945,  0.50996381,  0.3688674 ],
        ..., 
        [ 0.60144049,  0.3734293 ,  0.15790308],
        [ 0.59631842,  0.36886746,  0.14933962],
        [ 0.58831513,  0.36310509,  0.13717464]],

       [[ 0.46402639,  0.37791196,  0.22721158],
        [ 0.55430186,  0.46882766,  0.31364554],
        [ 0.59383738,  0.50548196,  0.35462168],
        ..., 
        [ 0.59839934,  0.38335338,  0.17038815],
        [ 0.58783519,  0.37366962,  0.15574239],
        [ 0.57727087,  0.36470595,  0.14229688]],

       ..., 
       [[ 0.63353324,  0.42016786,  0.21008389],
        